## Data Ingestion Stage

In [1]:
from collections import namedtuple
import os

In [19]:
#This is the entity
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_url",
    "local_file_path",
    "unzip_dir"
])

In [36]:
os.chdir("CNN_Class/")

In [37]:
pwd

'/Users/hrishikeshpoduval08/Desktop/ai_projects/CNN_Class'

In [38]:
ls

LICENSE           logs/             research/         template.py
README.md         params.yaml       setup.py          test.py
config/           requirements.txt  src/              tp.py


In [29]:
pwd

'/Users/hrishikeshpoduval08/Desktop/ai_projects/CNN_Class/src'

In [39]:
from cnn_classifier.constants import *
from cnn_classifier.utils import read_yaml,create_directories

In [40]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.root])
    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        dataingestion = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_file_path = config.local_file_path,
            unzip_dir = config.unzip_dir
        )
        return dataingestion


In [41]:
import os
import urllib.request as request
from zipfile import ZipFile

In [42]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_file_path):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_file_path
            )
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_file_path, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-03-18 13:02:33,909: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-18 13:02:33,911: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 13:02:33,912: INFO: common: created directory at: data]
[2024-03-18 13:02:33,913: INFO: common: created directory at: data/data_ingestion]
